In [53]:
import pickle
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [54]:
# # Load data from each band (assuming files are named accordingly)
# with open('new_data/pd_off vs hc/delta.pkl', 'rb') as f:
#     delta_data = pickle.load(f)['data']
# with open('new_data/pd_off vs hc/theta.pkl', 'rb') as f:
#     theta_data = pickle.load(f)['data']
# with open('new_data/pd_off vs hc/alpha.pkl', 'rb') as f:
#     alpha_data = pickle.load(f)['data']
# with open('new_data/pd_off vs hc/beta.pkl', 'rb') as f:
#     beta_data = pickle.load(f)['data']
# with open('new_data/pd_off vs hc/gamma.pkl', 'rb') as f:
#     gamma_data = pickle.load(f)['data']

# # Load labels (assume the labels are the same for each band)
# with open('new_data/pd_off vs hc/delta.pkl', 'rb') as f:
#     labels = pickle.load(f)['labels']


In [55]:
# Load data from each band (assuming files are named accordingly)
with open('IowaDataset/processed/delta.pkl', 'rb') as f:
    delta_data = pickle.load(f)['data']
with open('IowaDataset/processed/theta.pkl', 'rb') as f:
    theta_data = pickle.load(f)['data']
with open('IowaDataset/processed/alpha.pkl', 'rb') as f:
    alpha_data = pickle.load(f)['data']
with open('IowaDataset/processed/beta.pkl', 'rb') as f:
    beta_data = pickle.load(f)['data']
with open('IowaDataset/processed/gamma.pkl', 'rb') as f:
    gamma_data = pickle.load(f)['data']


# Load labels (assume the labels are the same for each band)
with open('IowaDataset/processed/gamma.pkl', 'rb') as f:
    labels = pickle.load(f)['labels']

In [56]:
data=delta_data

In [57]:
print(data.shape)
print(labels.shape)

(3360, 29)
(3360,)


In [58]:
data[7][0]

8.541414077285762e-12

In [59]:
import numpy as np
data = np.log1p(data * 1e15)  # log(1 + x) to prevent log(0)


In [60]:
data[7][0]

9.052798925675562

In [61]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, random_state=42, stratify=labels)

In [62]:
X_train.shape

(3024, 29)

In [63]:
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold

In [64]:
# Define K-Fold cross-validation
k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Convert data into NumPy arrays (ensure they are properly formatted)
X = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)  # Reshape for 1D-CNN
y = np.array(y_train)

# Store accuracy for each fold
fold_accuracies = []

# K-Fold Training Loop
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\nTraining Fold {fold + 1}/{k}...")

    # Split data
    X_train_fold, X_val_fold = X[train_idx], X[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    # Define a fresh model for each fold
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(29, 1)),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),

        Dense(1, activation='sigmoid')  # Sigmoid for binary classification
    ])

    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Train model
    history = model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, 
                        validation_data=(X_val_fold, y_val_fold), verbose=1)

    # Evaluate model on validation set
    val_loss, val_acc = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    print(f"Fold {fold + 1} Accuracy: {val_acc:.4f}")

    # Store accuracy
    fold_accuracies.append(val_acc)





Training Fold 1/5...
Epoch 1/10
76/76 [==============================] - 1s 7ms/step - loss: 0.7517 - accuracy: 0.5147 - val_loss: 0.6845 - val_accuracy: 0.5355
Epoch 2/10
76/76 [==============================] - 0s 5ms/step - loss: 0.6824 - accuracy: 0.5672 - val_loss: 0.6662 - val_accuracy: 0.6860
Epoch 3/10
76/76 [==============================] - 0s 6ms/step - loss: 0.6596 - accuracy: 0.6019 - val_loss: 0.6012 - val_accuracy: 0.7091
Epoch 4/10
76/76 [==============================] - 0s 5ms/step - loss: 0.5762 - accuracy: 0.6900 - val_loss: 0.4878 - val_accuracy: 0.7504
Epoch 5/10
76/76 [==============================] - 0s 4ms/step - loss: 0.5001 - accuracy: 0.7288 - val_loss: 0.4143 - val_accuracy: 0.8050
Epoch 6/10
76/76 [==============================] - 0s 5ms/step - loss: 0.4222 - accuracy: 0.7788 - val_loss: 0.3560 - val_accuracy: 0.8083
Epoch 7/10
76/76 [==============================] - 0s 5ms/step - loss: 0.3934 - accuracy: 0.8041 - val_loss: 0.3321 - val_accuracy: 0.839

In [65]:
# Print average accuracy
print(f"\nAverage Accuracy across {k} folds: {np.mean(fold_accuracies):.4f} ± {np.std(fold_accuracies):.4f}")


Average Accuracy across 5 folds: 0.8505 ± 0.0104
